### Downloading the data

In [ ]:
from main import main
key = '<replace with your key>' # this is the key for the weather data API

In [ ]:
# uncomment this line at your own risk
# it will take ~15-20 minutes to download the data and to do some cleaning
# main(key)

In [ ]:
from configs import merged_cycle_data_file
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV

### Load the weather & holiday data

In [ ]:
from configs import weather_data_csv

holidays = 'holidays.csv'
hol_df = pd.read_csv(holidays, index_col='date', parse_dates=['date'])
weather_df = pd.read_csv(weather_data_csv, index_col='timestamp', parse_dates=['timestamp'])
weather_df.head()

In [ ]:
# get a set of the holiday dates
hol_set= set(hol_df.index.map(lambda x: x.date()))

### Load the merged csv file by reading it in chunks

# Warning:
The next cell takes a lot of time (it took 5h 10 mins for us) so uncomment this cell at your own risk.

In [ ]:
# %%time
# from datetime import datetime
# import time

# chunk_size = 500000
# date_mapper = lambda x: pd.to_datetime(datetime(year=x.year, month=x.month, day=x.day, hour=x.hour))

# bike_share_df = pd.DataFrame()
# print('Started loading merged_cycle_data_file.')
# iter_ = pd.read_csv(merged_cycle_data_file, chunksize=chunk_size, iterator=True,
#         index_col='Rental Id',
#         parse_dates=['End Date', 'Start Date'])
# print('Finished loading merged_cycle_data_file.')

# r_start = time.time()
# for i, df in enumerate(iter_):
#     r_end = time.time()
#     print(f'{i+1}. Read rows {chunk_size*i}:{chunk_size*(i+1)} in {r_end-r_start:.3f}. ', end='')

#     start = time.time()
#     df = df.dropna()
#     # leave only entries that have valid duration
#     df = df[df['Duration'] > 0]
    
#     diff = df['End Date'] - df['Start Date'] # compute the difference between the objects
#     seconds = diff.map(lambda x: x.total_seconds()) # map to seconds
#     df = df[(df['Duration'] == seconds) & (seconds >= 0)] # check if duration matches the result and if the result is positive
    
    
#     # keep only year, month, day, hour information from the start date
#     df['Start Date'] = df['Start Date'].map(date_mapper)
#     # count the number of shares per hour
#     share_df = df.groupby('Start Date').agg({'Start Date': 'count'}).rename(columns={'Start Date': 'share_count'})
#     # join of weather data
#     share_df = share_df.join(weather_df)
#     # reset index
#     share_df = share_df.reset_index().rename(columns={'index': 'Start Date'})
#     share_df = share_df.dropna()
    
#     share_df['month'] = share_df['Start Date'].apply(lambda t: t.month)
#     share_df['weekday'] = share_df['Start Date'].apply(lambda t: t.weekday())
#     share_df['hour'] = share_df['Start Date'].apply(lambda t: t.hour)
#     share_df['is_holiday'] = share_df['Start Date'].map(lambda x: x.date() in hol_set).map(lambda x: '1' if x else '0')
#     # check if start date hits on a weekend
#     # monday is 0, sunday is 6
#     share_df['is_weekend'] = share_df['Start Date'].map(lambda x: x.weekday() > 4).map(lambda x: '1' if x else '0')
#     share_df['weatherCode'] = share_df['weatherCode'].map(lambda x: str(int(x)))
    
#     bike_share_df = bike_share_df.append(share_df)
#     end = time.time()
#     print(f'Completed cleaning & merging in {end-start:3.3f} seconds.')
#     r_start = time.time()

# print('Finished reading!')
# # bike_share_df = bike_share_df.reset_index(drop=True)
# # because of reading in chunks, some hours appear multiple times
# print('Grouping the hours')
# share_total = bike_share_df.groupby('Start Date').agg({'share_count': 'sum'})
# share_row = bike_share_df.groupby('Start Date').agg(lambda x: x.iloc[0])
# share_row['share_count'] = share_total['share_count']
# bike_share_df = share_row.reset_index()
# print('Saving to file')
# # save the data to a file, so that we can load it faster next time
# bike_share_df.sort_values('Start Date').reset_index(drop=True).to_csv('bike-shares.csv', index=False)

In [ ]:
types = {
    'weatherCode': str,
    'is_holiday': str,
    'is_weekend': str
}
bike_share_df = pd.read_csv('bike-shares.csv', parse_dates=['Start Date'], dtype=types)
fig_size = (18, 25)
random_state = 42069
test_size = 0.2

In [ ]:
bike_share_df.info()

In [ ]:
bike_share_df.head()

In [ ]:
bike_share_df, test_set = train_test_split(bike_share_df, test_size=test_size, random_state=random_state)

In [ ]:
bike_share_df = bike_share_df.sort_values('Start Date')

In [ ]:
bike_share_df.describe()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
import numpy as np
# get the right column indices: safer than hard-coding indices 3, 4, 5, 6
weather_ix, hour_ix, month_ix, share_ix = [
    list(bike_share_df.columns).index(col)
    for col in ('weatherCode', 'hour', 'month', 'share_count')]

cat_attribs = ['weatherCode', 'is_holiday', 'is_weekend']
num_attribs = list(bike_share_df.drop(cat_attribs, axis=1))

# num_pipeline = Pipeline([
#     ('selector', DataFrameSelector(num_attribs)),
#     ('imputer', Imputer(strategy="median")),
#     ('attribs_adder', CombinedAttributesAdder()),
#     ('std_scaler', StandardScaler()),
# ])


# cat_pipeline = Pipeline([
#     ('selector', DataFrameSelector(cat_attribs)),
#     ('label_binarizer', LabelBinarizer()),
# ])



def group_weather_codes(X):
    new_codes = {
        '113': 'Clear', # Clear, Sunny
        '116': 'Cloudy', # Partly cloudy
        '122': 'Cloudy', # Overcast
        '119': 'Cloudy', # Cloudy
        '143': 'Lowered visibility', # Mist
        '176': 'Light rain', # Patchy rain possible
        '296': 'Light rain', # Light rain
        '308': 'Heavy rain', # Heavy rain
        '302': 'Heavy rain', # Moderate rain
        '353': 'Light rain', # Light rain shower
        '266': 'Light rain', # Light drizzle
        '356': 'Heavy rain', # Moderate or heavy rain shower
        '293': 'Light rain', # Patchy light rain
        '248': 'Lowered visibility', # Fog
        '200': 'Thunder', # Thundery outbreaks possible
        '299': 'Heavy rain', # Moderate rain at times
        '263': 'Light rain', # Patchy light drizzle
        '317': 'Light snow', # Light sleet
        '386': 'Thunder', # Patchy light rain with thunder
        '230': 'Blizzard', # Blizzard
        '329': 'Heavy snow', # Patchy moderate snow
        '332': 'Heavy snow', # Moderate snow
        '338': 'Heavy snow', # Heavy snow
        '326': 'Light snow', # Light snow
        '362': 'Light snow', # Light sleet showers
        '389': 'Thunder', # Moderate or heavy rain with thunder
        '311': 'Light snow', # Light freezing rain
        '359': 'Heavy rain', # Torrential rain shower
        '323': 'Light snow', # Patchy light snow
        '260': 'Lowered visibility', # Freezing fog
        '368': 'Light snow', # Light snow showers
        '371': 'Heavy snow', # Moderate or heavy snow showers
        '305': 'Heavy rain', # Heavy rain at times
        '320': 'Heavy snow', # Moderate or heavy sleet
        '335': 'Heavy snow', # Patchy heavy snow
        '227': 'Heavy snow', # Blowing snow
    }
    weather_codes = X[:, weather_ix].flatten() # each item in the array is a list, so flatten to one single list
    new_codes = np.array([new_codes[code] for code in weather_codes], dtype=str)

    X = np.c_[X]
    X[:, weather_ix] = new_codes
    return X

def convert_cyclical_data(X, convert_hours=True, convert_months=True):
    to_remove = []
    if convert_hours:
        hours = X[:, hour_ix].flatten().astype(float)

        hr_sin = np.sin(hours * (2.*np.pi/24))
        hr_cos = np.cos(hours * (2.*np.pi/24))
        to_remove.append(hour_ix)
        X = np.c_[X, hr_sin, hr_cos]
    if convert_months:
        months = (X[:, month_ix] - 1).flatten().astype(float) # shift to 0..11

        mnth_sin = np.sin(months * (2.*np.pi/12))
        mnth_cos = np.cos(months * (2.*np.pi/12))
        to_remove.append(month_ix)
        X = np.c_[X, mnth_sin, mnth_cos]

    X = np.delete(X, to_remove, 1)
    return np.c_[X]

def add_lag_value(X, count=1):
    # assume x is already sorted
    for i in range(1, count+1):
        X = np.c_[X, np.roll(X[:, share_ix], i)]
#     return X[count:]
    return X

# full_pipeline = ColumnTransformer([
#     ('weather_grouper', FunctionTransformer(group_weather_codes, validate=False), [weather_ix]),
#     ('cyclical_transformer', FunctionTransformer(convert_cyclical_data, validate=False), [hour_ix, month_ix]),
#     ('lag_value_appender', FunctionTransformer(add_lag_value, validate=False), [share_ix])
# ], remainder='passthrough')

In [ ]:
# shares_extra_attribs = full_pipeline.fit_transform(bike_share_df.values)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
bike_share_df.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
bike_share_df['month'].hist(bins=12)

In [ ]:
bike_share_df['weatherCode'].value_counts()

In [ ]:
bike_share_df['hour'].hist(bins=24)

In [ ]:
bike_share_df['temperature'].hist(bins=24)

In [ ]:
bike_share_df[bike_share_df['share_count'] <= 100].shape

In [ ]:
bike_share_df['share_count'].hist(bins=200, figsize=(24,24))

In [ ]:
bike_share_df[bike_share_df['share_count'] <= 300]['share_count'].hist(bins=100, figsize=(24,24))

In [ ]:
shares_extra_attribs = group_weather_codes(bike_share_df.values)
shares_extra_attribs = pd.DataFrame(
    shares_extra_attribs,
    columns=bike_share_df.columns,
    index=bike_share_df.index)

In [ ]:
shares_extra_attribs['weatherCode'].hist()

In [ ]:
fig,(ax1, ax2, ax3)= plt.subplots(nrows=3)
fig.set_size_inches(fig_size)

sns.pointplot(data=shares_extra_attribs, x='weatherCode', y='share_count', ax=ax1, join=False)
sns.pointplot(data=shares_extra_attribs, x='hour', y='share_count', hue='is_holiday', ax=ax2)
sns.pointplot(data=shares_extra_attribs, x='hour', y='share_count', hue='is_weekend', ax=ax3)
ax1.set_xlabel('Weather code', labelpad=30)
_ = ax1.set_ylabel('Share count', labelpad=15)

### Dealing with cyclical data

In [ ]:
shares_extra_attribs = convert_cyclical_data(shares_extra_attribs.values)

shares_extra_attribs = pd.DataFrame(
    shares_extra_attribs,
    columns=list(bike_share_df.columns.drop(['hour', 'month']))+['hr_sin', 'hr_cos', 'mnth_sin', 'mnth_cos'],
    index=bike_share_df.index)

In [ ]:
fig, ax= plt.subplots(nrows=1)
fig.set_size_inches(15,10)

pearsoncorr = bike_share_df.drop('Start Date', axis=1).corr(method='pearson')
heat_map=sns.heatmap(pearsoncorr, 
            xticklabels=pearsoncorr.columns,
            yticklabels=pearsoncorr.columns,
            cmap='RdBu_r',
            annot=True,
            linewidth=0.5,
           ax=ax)
_ = heat_map.set_yticklabels(heat_map.get_yticklabels(), rotation=0)

### Adding lag value

In [ ]:
count=3
shares_extra_attribs_ = add_lag_value(shares_extra_attribs.values, count)
shares_extra_attribs = pd.DataFrame(
    shares_extra_attribs_,
    columns=list(shares_extra_attribs.columns)+['lag_1', 'lag_2', 'lag_3'],
    index=bike_share_df.index)
shares_extra_attribs = shares_extra_attribs[count:]

In [ ]:
fig, ax= plt.subplots(nrows=1)
fig.set_size_inches(15,10)

pearsoncorr = shares_extra_attribs.drop('Start Date', axis=1).corr(method='pearson')
heat_map=sns.heatmap(pearsoncorr, 
            xticklabels=pearsoncorr.columns,
            yticklabels=pearsoncorr.columns,
#             cmap='RdBu_r',
#             annot=True,
            linewidth=0.5,
            ax=ax)
_ = heat_map.set_yticklabels(heat_map.get_yticklabels(), rotation=0)

In [ ]:
shares_extra_attribs = group_weather_codes(bike_share_df.values)
shares_extra_attribs = convert_cyclical_data(shares_extra_attribs)
shares_extra_attribs = add_lag_value(shares_extra_attribs)
shares_extra_attribs = pd.DataFrame(
    shares_extra_attribs,
    columns=list(bike_share_df.columns.drop(['hour', 'month']))+['hr_sin', 'hr_cos', 'mnth_sin', 'mnth_cos', 'lag_1'],
    index=bike_share_df.index)
shares_extra_attribs.head()

### Deal with labeled data

In [ ]:
encoded_weather_codes = pd.get_dummies(bike_share_df['weatherCode'], prefix='weather')
bike_share_df_enc = bike_share_df.drop(['weatherCode'], axis=1)
bike_share_df_enc = bike_share_df_enc.join(encoded_weather_codes)

In [ ]:
holiday_enc = pd.get_dummies(bike_share_df['is_holiday'], prefix='holiday')
week_enc = pd.get_dummies(bike_share_df['is_weekend'], prefix='weekend')
bike_share_df_enc = bike_share_df_enc.join(holiday_enc).join(week_enc)
bike_share_df_enc = bike_share_df_enc.drop(['is_holiday', 'is_weekend'], axis=1)
# "weekday" could be dropped due to the fact that we have "is_weekend"
bike_share_df_enc = bike_share_df_enc.drop(['weekday'], axis=1)

In [ ]:
fig, ax= plt.subplots(nrows=1)
fig.set_size_inches(fig_size)

pearsoncorr = bike_share_df_enc.drop('Start Date', axis=1).corr(method='pearson')
heat_map=sns.heatmap(pearsoncorr, 
            xticklabels=pearsoncorr.columns,
            yticklabels=pearsoncorr.columns,
            cmap='RdBu_r',
            annot=True,
            linewidth=0.5,
           ax=ax)
_ = heat_map.set_yticklabels(heat_map.get_yticklabels(), rotation=0)

### Training the model

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, SVR
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import Lasso, ElasticNet, Ridge, SGDRegressor

In [ ]:
# shuffle the dataframe
from sklearn.utils import shuffle
bike_share_df = shuffle(bike_share_df, random_state=random_state).reset_index(drop=True)
bike_share_df

In [ ]:
train_set = bike_share_df_enc.drop('Start Date', axis=1)
test_set = test_set.drop('Start Date', axis=1)

X_train = train_set.drop('share_count', axis=1).to_numpy()
Y_train = train_set['share_count'].to_numpy()

X_test = test_set.drop('share_count', axis=1).to_numpy()
Y_test = test_set['share_count'].to_numpy()

In [ ]:
classifiers = [
#     ('Decision Tree', DecisionTreeClassifier()),
#     ('Random Forest', RandomForestClassifier(n_estimators=100)),
#     ('NN (50, 50, 16)', MLPClassifier(hidden_layer_sizes=(16, 16, 16), verbose=True, learning_rate='adaptive', activation='tanh', learning_rate_init=0.01))
    ('Lasso', Lasso(random_state=random_state)),
    ('ElasticNet', ElasticNet(random_state=random_state)),
    ('Ridge', Ridge(random_state=random_state)),
#     ('SVR linear', SVR(kernel='linear', verbose=True)),
#     ('SVR rbf', SVR(kernel='rbf', verbose=True)),
#     ('SGDRegressor', SGDRegressor()),
    ('GradientBoostRegressor', GradientBoostingRegressor()),
    ('GradientBoostRegressor(ls)', GradientBoostingRegressor(max_features=21, random_state=random_state, max_depth=8)),
    ('Optimized GradientBoostRegressor(huber)', GradientBoostingRegressor(loss='huber', max_features=21, n_estimators=160, random_state=random_state, max_depth=8))
]

In [ ]:
import time
def learn(classifiers, X_train, Y_train, X_test, Y_test):
    for name, clf in classifiers:
        print(f'** {name}')
        t0 = time.time()
        clf.fit(X_train, Y_train)
        t1 = time.time()
        score_train = clf.score(X_train[:10000], Y_train[:10000])
        t2 = time.time()
        score_test = clf.score(X_test, Y_test)
        t3 = time.time()
        print(f'\tTraining time {t1 - t0:3.3f}')
        print(f'\tPrediction time (train) {t2 - t1:3.3f}')
        print(f'\tPrediction time (test) {t3 - t2:3.3f}')
        print(f'\tScore train: {score_train:.3f}\tScore Test: {score_test:.3f}')

In [ ]:
# learn(classifiers, X_train, Y_train, X_test, Y_test)

In [ ]:
# # Search through param_grid parameters, find the best set of parameters for GBR (Gradient Boosting Regressor)
# param_grid = [
#     {'max_features': [21], 'max_depth': [3, 10, 20, 50, 100]}
#   ]

# clf = GradientBoostingRegressor(random_state=random_state)

# grid_search = GridSearchCV(clf, param_grid, cv=5,
#                            scoring='neg_mean_squared_error', return_train_score=True)
# grid_search.fit(X_train, Y_train)

# # Run GBR with best set of parameters
# clf = GradientBoostingRegressor(random_state=random_state,
#                                 **grid_search.best_params_,
#                                 n_estimators=2000)

# clf.fit(X_train, Y_train)
# results = clf.predict(X_test)
# score = clf.score(X_test, Y_test)
# print("TEST SCORE: " + str(score))

In [ ]:
optimized_clf = GradientBoostingRegressor(loss='huber', max_features=23, n_estimators=160, random_state=random_state, max_depth=8)
optimized_clf.fit(X_train, Y_train)
results = optimized_clf.predict(X_test)

score = optimized_clf.score(X_test, Y_test)
print("Test score: " + str(score))

In [ ]:
from sklearn.metrics import mean_squared_error

lin_mse = mean_squared_error(Y_test, results)
lin_rmse = np.sqrt(lin_mse)

In [ ]:
from sklearn.metrics import mean_absolute_error

lin_mae = mean_absolute_error(Y_test, results)

In [ ]:
# count negative
(results < 0).shape

In [ ]:
from sklearn.metrics import explained_variance_score
y_true = Y_test
y_pred = results
var_score = explained_variance_score(y_true, y_pred)  

In [ ]:
print("Mean squared error: " + str(lin_rmse))
print("Mean squared error: " + str(lin_mae))
print("Variance score: " + str(var_score))